# Federated PyTorch UNET Tutorial

In [1]:
# Install dependencies if not already installed
!pip install torch

You should consider upgrading via the '/home/itrushkin/.virtualenvs/openfl_research/bin/python -m pip install --upgrade pip' command.


First of all we need to set up our OpenFL workspace. To do this, simply run the `fx.init()` command as follows:

In [2]:
import openfl.native as fx

# Setup default workspace, logging, etc. Install additional requirements
fx.init('torch_unet_kvasir')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
├── agg_to_col_two_signed_cert.zip
├── data
│   └── MNIST
│       ├── raw
│       └── processed
├── cert
│   ├── ca
│   │   ├── signing-ca.crt
│   │   ├── root-ca.crt
│   │   ├── signing-ca
│   │   ├── signing-ca.csr
│   │   └── root-ca
│   ├── server
│   │   ├── agg_nnlicv611.inn.intel.com.csr
│   │   ├── agg_nnlicv611.inn.intel.com.key
│   │   └── agg_nnlicv611.inn.intel.com.crt
│   ├── client
│   │   ├── col_two.crt
│   │   ├── col_one.crt
│   │   ├── col_one.key
│   │   └── col_

In [3]:
# Import installed modules
import PIL
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from skimage import io
from torchvision import transforms as tsf
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from hashlib import sha384
from os import path
from os import listdir

from openfl.federated import FederatedModel, FederatedDataSet
from openfl.utilities import TensorKey

Download Kvasir dataset

In [4]:
!wget 'https://datasets.simula.no/hyper-kvasir/hyper-kvasir-segmented-images.zip' -O kvasir.zip
ZIP_SHA384 = 'e30d18a772c6520476e55b610a4db457237f151e'\
    '19182849d54b49ae24699881c1e18e0961f77642be900450ef8b22e7'
assert sha384(open('./kvasir.zip', 'rb').read(
    path.getsize('./kvasir.zip'))).hexdigest() == ZIP_SHA384
!unzip -n kvasir.zip -d ./data

--2021-04-29 17:42:32--  https://datasets.simula.no/hyper-kvasir/hyper-kvasir-segmented-images.zip
Resolving proxy-dmz.intel.com (proxy-dmz.intel.com)... 10.102.248.16
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.102.248.16|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 46251583 (44M) [application/zip]
Saving to: ‘kvasir.zip’

kvasir.zip          100%[===================>]  44,11M   514KB/s    in 98s     

2021-04-29 17:44:11 (460 KB/s) - ‘kvasir.zip’ saved [46251583/46251583]

Archive:  kvasir.zip
   creating: ./data/segmented-images/
  inflating: ./data/segmented-images/bounding-boxes.json  
   creating: ./data/segmented-images/masks/
  inflating: ./data/segmented-images/masks/cju88oh0po9gq0801nge4tgr1.jpg  
  inflating: ./data/segmented-images/masks/cju34uhepd3dd0799hs8782ad.jpg  
  inflating: ./data/segmented-images/masks/cju8bgdmqrksy0801tozdmraa.jpg  
  inflating: ./data/segmented-images/masks/cju5ddda9bkkt0850enzwatb1.jpg  
  inflating: 

  inflating: ./data/segmented-images/masks/cju85qefyln6v0850szeb9byi.jpg  
  inflating: ./data/segmented-images/masks/cju6vgdmivcvb08018fra5lnv.jpg  
  inflating: ./data/segmented-images/masks/cju8b8yair65w09878pyqtr96.jpg  
  inflating: ./data/segmented-images/masks/ck2395w2mb4vu07480otsu6tw.jpg  
  inflating: ./data/segmented-images/masks/cju2nguelpmlj0835rojdn097.jpg  
  inflating: ./data/segmented-images/masks/cjyzkmjy8evns070165gf9dmq.jpg  
  inflating: ./data/segmented-images/masks/cju0s2a9ekvms080138tjjpxr.jpg  
  inflating: ./data/segmented-images/masks/cju2rlqdnoz9k0993cpjae3x0.jpg  
  inflating: ./data/segmented-images/masks/cju7es23b2vcp0755gpbm9s7v.jpg  
  inflating: ./data/segmented-images/masks/cju5tenjojp1j0755ms4949h2.jpg  
  inflating: ./data/segmented-images/masks/cju7fq7mm2pw508176uk5ugtx.jpg  
  inflating: ./data/segmented-images/masks/cju83mki1jv5w0817kubxm31r.jpg  
  inflating: ./data/segmented-images/masks/cju5c5xc7algd0817pb1ej5yo.jpg  
  inflating: ./data/segme

  inflating: ./data/segmented-images/images/cju6us80mv1b50871ebyq2wxa.jpg  
  inflating: ./data/segmented-images/images/cju87vqa0ndwg0850onjdz7ol.jpg  
  inflating: ./data/segmented-images/images/cju2sxf3iqbpv09937iksn8ep.jpg  
  inflating: ./data/segmented-images/images/cju5wtdu4m0im0871mix0yvc0.jpg  
  inflating: ./data/segmented-images/images/cju8ando2qqdo0818ck7i1be1.jpg  
  inflating: ./data/segmented-images/images/cju85plp7lmkw0850rx42jdpf.jpg  
  inflating: ./data/segmented-images/images/cju2u4pymvc720988wsxrmi84.jpg  
  inflating: ./data/segmented-images/images/cju35k2fr3vc50988c85qkrwg.jpg  
  inflating: ./data/segmented-images/images/cju2qz06823a40878ojcz9ccx.jpg  
  inflating: ./data/segmented-images/images/cju324q101fhe08350wae9cif.jpg  
  inflating: ./data/segmented-images/images/cju42u5bjlvi10801dc13sskp.jpg  
  inflating: ./data/segmented-images/images/cju7bb3ss1uoo0755pmhyco7t.jpg  
  inflating: ./data/segmented-images/images/cju2oo0wh0bqy0878biujeyhe.jpg  
  inflating:

  inflating: ./data/segmented-images/images/cju8b8yair65w09878pyqtr96.jpg  
  inflating: ./data/segmented-images/images/cju7cp6dw244p0818gncdol4m.jpg  
  inflating: ./data/segmented-images/images/cju88aq6vo1ij0755c2ey7z7n.jpg  
  inflating: ./data/segmented-images/images/cju3v664kh0px0818y4y7wolf.jpg  
  inflating: ./data/segmented-images/images/cju5x15djm7ae0755h8czf6nt.jpg  
  inflating: ./data/segmented-images/images/cju88l66no10s0850rsda7ej1.jpg  
  inflating: ./data/segmented-images/images/cju6ywm40wdbo0987pbftsvtg.jpg  
  inflating: ./data/segmented-images/images/cju17r8il13910799dr2wme2e.jpg  
  inflating: ./data/segmented-images/images/cju7cufm7298k0755j09uf3of.jpg  
  inflating: ./data/segmented-images/images/cju2xf8e5y2wm08359vcgk09b.jpg  
  inflating: ./data/segmented-images/images/cju7g7ba42z310987bqzbi2bq.jpg  
  inflating: ./data/segmented-images/images/cju3ul8dogf1z09872y2ecowp.jpg  
  inflating: ./data/segmented-images/images/cju33eqwbcch208012jikwdky.jpg  
  inflating:

  inflating: ./data/segmented-images/images/cju8abobpqbir08189u01huru.jpg  
  inflating: ./data/segmented-images/images/cju8axq24r4an0755yhv9d4ly.jpg  
  inflating: ./data/segmented-images/images/cju32csyfblyh080170aa3x5p.jpg  
  inflating: ./data/segmented-images/images/cju76erapykj30871x5eaxh4q.jpg  
  inflating: ./data/segmented-images/images/cju77vvcwzcm50850lzoykuva.jpg  
  inflating: ./data/segmented-images/images/cju30df2j09dd08351ayx2t6w.jpg  
  inflating: ./data/segmented-images/images/cju43c92lm5cj0755lorsorfg.jpg  
  inflating: ./data/segmented-images/images/cju5vxuc5loxw0818u8xgf45p.jpg  
  inflating: ./data/segmented-images/images/cju0t4oil7vzk099370nun5h9.jpg  
  inflating: ./data/segmented-images/images/cju886ryxnsl50801r93jai7q.jpg  
  inflating: ./data/segmented-images/images/cju5eq8c8ck690850vix98hv3.jpg  
  inflating: ./data/segmented-images/images/cju5cjh3xattc0817j2vbulzi.jpg  
  inflating: ./data/segmented-images/images/cju1fb9236a110801yvg0fwju.jpg  
  inflating:

  inflating: ./data/segmented-images/images/cju14pxbaoksp0835qzorx6g6.jpg  
  inflating: ./data/segmented-images/images/cju5bj926aiec07559rshy4wa.jpg  
  inflating: ./data/segmented-images/images/cju2yi9tz8vky0801yqip0xyl.jpg  
  inflating: ./data/segmented-images/images/cju7fazv92ywx0755xov2erga.jpg  
  inflating: ./data/segmented-images/images/cju8aqq8uqmoq0987hphto9gg.jpg  
  inflating: ./data/segmented-images/images/cju40jl7skiuo0817p0smlgg8.jpg  
  inflating: ./data/segmented-images/images/cju2igw4gvxds0878808qj398.jpg  
  inflating: ./data/segmented-images/images/cju5ekty5ckzf07550c9u3ckk.jpg  
  inflating: ./data/segmented-images/images/cju5wcc90lu020850mjrxppv6.jpg  
  inflating: ./data/segmented-images/images/cju7druhp2gp308715i6km7be.jpg  
  inflating: ./data/segmented-images/images/cju8914beokbf0850isxpocrk.jpg  
  inflating: ./data/segmented-images/images/cju7ehljc2or70871261br8ai.jpg  
  inflating: ./data/segmented-images/images/cju83yddek68q0850d2x7zfkm.jpg  
  inflating:

  inflating: ./data/segmented-images/images/cju8dn0c3u2v50801k8rvq02f.jpg  
  inflating: ./data/segmented-images/images/cju3x4blzieu30850x10uuvbm.jpg  
  inflating: ./data/segmented-images/images/cjyzkmjy8evns070165gf9dmq.jpg  
  inflating: ./data/segmented-images/images/cju40wto5kxwi0755it190f2k.jpg  
  inflating: ./data/segmented-images/images/cjyztzaqtrv430848l8xgcerw.jpg  
  inflating: ./data/segmented-images/images/cjyzurzvohqnr0794es1itzek.jpg  
  inflating: ./data/segmented-images/images/cju2oi8sq0i2y0801mektzvw8.jpg  
  inflating: ./data/segmented-images/images/cju7eea9b2m0z0801ynqv1fqu.jpg  
  inflating: ./data/segmented-images/images/cju8dqkrqu83i0818ev74qpxq.jpg  
  inflating: ./data/segmented-images/images/cjyzkpsbjdsjq07211dfi4sru.jpg  
  inflating: ./data/segmented-images/images/cjyzk8qieoboa0848ogj51wwm.jpg  
  inflating: ./data/segmented-images/images/cjyzu3reghjya0794w7pwoi50.jpg  
  inflating: ./data/segmented-images/images/cjyzufihqquiw0a46jatrbwln.jpg  
  inflating:

Now we are ready to define our dataset and model to perform federated learning on.

In [5]:
DATA_PATH = './data/segmented-images/'

In [6]:
def read_data(image_path, mask_path):
    """
    Read image and mask from disk.
    """
    img = io.imread(image_path)
    assert(img.shape[2] == 3)
    mask = io.imread(mask_path)
    return (img, mask[:, :, 0].astype(np.uint8))


class KvasirDataset(Dataset):
    """
    Kvasir dataset contains 1000 images for all collaborators.
    Args:
        data_path: path to dataset on disk
        collaborator_count: total number of collaborators
        collaborator_num: number of current collaborator
        is_validation: validation option
    """

    def __init__(self, data_path, collaborator_count, collaborator_num, is_validation):
        self.images_path = './data/segmented-images/images/'
        self.masks_path = './data/segmented-images/masks/'
        self.images_names = [img_name for img_name in sorted(listdir(
            self.images_path)) if len(img_name) > 3 and img_name[-3:] == 'jpg']

        self.images_names = self.images_names[collaborator_num:: collaborator_count]
        self.is_validation = is_validation
        assert(len(self.images_names) > 8)
        validation_size = len(self.images_names) // 8
        if is_validation:
            self.images_names = self.images_names[-validation_size:]
        else:
            self.images_names = self.images_names[: -validation_size]

        self.img_trans = tsf.Compose([
            tsf.ToPILImage(),
            tsf.Resize((332, 332)),
            tsf.ToTensor(),
            tsf.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
        self.mask_trans = tsf.Compose([
            tsf.ToPILImage(),
            tsf.Resize((332, 332), interpolation=PIL.Image.NEAREST),
            tsf.ToTensor()])

    def __getitem__(self, index):
        name = self.images_names[index]
        img, mask = read_data(self.images_path + name, self.masks_path + name)
        img = self.img_trans(img).numpy()
        mask = self.mask_trans(mask).numpy()
        return img, mask

    def __len__(self):
        return len(self.images_names)

Here we redefine `FederatedDataSet` methods, if we don't want to use default batch generator from `FederatedDataSet`. 

In [7]:
class KvasirFederatedDataset(FederatedDataSet):
    def __init__(self, collaborator_count=1, collaborator_num=0, batch_size=1, **kwargs):
        """Instantiate the data object
        Args:
            collaborator_count: total number of collaborators
            collaborator_num: number of current collaborator
            batch_size:  the batch size of the data loader
            **kwargs: additional arguments, passed to super init
        """
        super().__init__([], [], [], [], batch_size, num_classes=2, **kwargs)

        self.collaborator_num = int(collaborator_num)

        self.batch_size = batch_size

        self.training_set = KvasirDataset(
            DATA_PATH, collaborator_count, collaborator_num, is_validation=False
        )
        self.valid_set = KvasirDataset(
            DATA_PATH, collaborator_count, collaborator_num, is_validation=True
        )

        self.train_loader = self.get_train_loader()
        self.val_loader = self.get_valid_loader()

    def get_valid_loader(self, num_batches=None):
        return DataLoader(self.valid_set, num_workers=8, batch_size=self.batch_size)

    def get_train_loader(self, num_batches=None):
        return DataLoader(
            self.training_set, num_workers=8, batch_size=self.batch_size, shuffle=True
        )

    def get_train_data_size(self):
        return len(self.training_set)

    def get_valid_data_size(self):
        return len(self.valid_set)

    def get_feature_shape(self):
        return self.valid_set[0][0].shape

    def split(self, collaborator_count, shuffle=True, equally=True):
        return [
            KvasirFederatedDataset(collaborator_count,
                           collaborator_num, self.batch_size)
            for collaborator_num in range(collaborator_count)
        ]

Our Unet model

In [8]:
def soft_dice_loss(output, target):
    num = target.size(0)
    m1 = output.view(num, -1)
    m2 = target.view(num, -1)
    intersection = m1 * m2
    score = 2.0 * (intersection.sum(1) + 1) / (m1.sum(1) + m2.sum(1) + 1)
    score = 1 - score.sum() / num
    return score


def soft_dice_coef(output, target):
    num = target.size(0)
    m1 = output.view(num, -1)
    m2 = target.view(num, -1)
    intersection = m1 * m2
    score = 2.0 * (intersection.sum(1) + 1) / (m1.sum(1) + m2.sum(1) + 1)
    return score.sum()


class double_conv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.in_ch = in_ch
        self.out_ch = out_ch
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2), double_conv(in_ch, out_ch))

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        self.in_ch = in_ch
        self.out_ch = out_ch
        if bilinear:
            self.up = nn.Upsample(
                scale_factor=2, mode="bilinear", align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(
                in_ch, in_ch // 2, 2, stride=2
            )
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX //
                        2, diffY // 2, diffY - diffY // 2))

        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        self.inc = double_conv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 1024)
        self.up1 = up(1024, 512)
        self.up2 = up(512, 256)
        self.up3 = up(256, 128)
        self.up4 = up(128, 64)
        self.outc = nn.Conv2d(64, n_classes, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        x = torch.sigmoid(x)
        return x

    def validate(
        self, col_name, round_num, input_tensor_dict, use_tqdm=False, **kwargs
    ):
        """ Validate. Redifine function from PyTorchTaskRunner, to use our validation"""
        self.rebuild_model(round_num, input_tensor_dict, validation=True)
        self.eval()
        self.to(self.device)
        val_score = 0
        total_samples = 0

        loader = self.data_loader.get_valid_loader()
        if use_tqdm:
            loader = tqdm.tqdm(loader, desc="validate")

        with torch.no_grad():
            for data, target in loader:
                samples = target.shape[0]
                total_samples += samples
                data, target = (
                    torch.tensor(data).to(self.device),
                    torch.tensor(target).to(self.device),
                )
                output = self(data)
                # get the index of the max log-probability
                val = soft_dice_coef(output, target)
                val_score += val.sum().cpu().numpy()

        origin = col_name
        suffix = "validate"
        if kwargs["apply"] == "local":
            suffix += "_local"
        else:
            suffix += "_agg"
        tags = ("metric", suffix)
        output_tensor_dict = {
            TensorKey("dice_coef", origin, round_num, True, tags): np.array(
                val_score / total_samples
            )
        }
        return output_tensor_dict, {}


def optimizer(x): return optim.Adam(x, lr=1e-3)

Create `KvasirFederatedDataset`, federated datasets for collaborators will be created in `split()` method of this object

In [9]:
fl_data = KvasirFederatedDataset(batch_size=6)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with OpenFL. It provides built-in federated training function which will be used while training. Using its `setup` function, collaborator models and datasets can be automatically obtained for the experiment. 

In [10]:
# Create a federated model using the pytorch class, optimizer function, and loss function
fl_model = FederatedModel(build_model=UNet, optimizer=optimizer,
                          loss_fn=soft_dice_loss, data_loader=fl_data)

In [11]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one': collaborator_models[0], 'two': collaborator_models[1]}

We can see the current FL plan values by running the `fx.get_plan()` function

In [12]:
# Get the current values of the FL plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_unet_kvasir_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_unet_kvasir_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_unet_kvasir_last.pbuf",
    "aggregator.settings.rounds_to_train": 40,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.epochs_per_round": 1.0,
    "collaborator.settings.opt_treatment": "RESET",
    "collaborator.settings.polling_interval":

Now we are ready to run our experiment. If we want to pass in custom FL plan settings, we can easily do that with the `override_config` parameter

In [13]:
# Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(
    collaborators, override_config={'aggregator.settings.rounds_to_train': 30})

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:287: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
<ipython-input-8-5e98da9ec23c>:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(data).to(self.device),
<ipython-input-8-5e98da9ec23c>:125: UserWarning: To copy construct from a tensor, it is recommended

/home/itrushkin/.virtualenvs/openfl_research/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:154: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


KeyboardInterrupt: 

In [ ]:
# Save final model
final_fl_model.save_native('final_pytorch_model')

Let's visually evaluate the results

In [ ]:
collaborator = collaborator_models[0]
loader = collaborator.runner.data_loader.get_valid_loader()
model = final_fl_model.model
model.eval()
device = final_fl_model.runner.device
model.to(device)
with torch.no_grad():
    for batch, _ in zip(loader, range(5)):
        preds = model(batch[0].to(device))
        for image, pred, target in zip(batch[0], preds, batch[1]):
            plt.figure(figsize=(10, 10))
            plt.subplot(131)
            plt.imshow(image.permute(1, 2, 0).data.cpu().numpy() * 0.5 + 0.5)
            plt.title("img")
            plt.subplot(132)
            plt.imshow(pred[0].data.cpu().numpy())
            plt.title("pred")
            plt.subplot(133)
            plt.imshow(target[0].data.cpu().numpy())
            plt.title("targ")
            plt.show()